# Extracting a single channel

Written by Simon M. Mudd, updated 27/03/2023

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **Download some data** section. 

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **First get data** section. 

First we install `lsdviztools`. 
This will take around a minute.

In [ ]:
!pip install lsdviztools &> /dev/null

Now we need to install lsdtopotools. We do this using something called `mamba`. Note that this version of `mamba` works for python 3.9 (which is what google colab currently uses).

This step will take around 20 seconds. 

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Mambaforge-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh &> /dev/null
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX &> /dev/null

Alternatively we can do this with condacolab, but this broke in March 2023 so will take some time to be fixed

In [ ]:
#!pip install -q condacolab
#import condacolab
#condacolab.install()

Now use mamba to install `lsdtopotools`. 
This step takes a bit over a minute. 

In [ ]:
!mamba install -y lsdtopotools &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. 

In [ ]:
!lsdtt-basic-metrics

## Get the DEM

Grab the DEM from opentopography.org

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw

In [ ]:
# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "Diablo"
source_name = "COP30"

SB_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = [35.1920215020742,-120.90387764783046], 
                        upper_right_coordinates = [35.296562615076155, -120.73735491824398],
                        prefix = Dataset_prefix, 
                        api_key_file = your_OT_api_key_file)
SB_DEM.print_parameters()
SB_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

# Get the point from which to extract the channel

In [ ]:
import pandas as pd

In [ ]:
d = {'id': [0], 'latitude': [35.25298220408284], 'longitude': [-120.77594126937667]}
df = pd.DataFrame(data=d)

Print this to a csv file

In [ ]:
df.to_csv("channel_source.csv",index=False)

# Set up parameters for an *lsdtopotools* run

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true", 
                    "extract_single_channel" : "true", 
                    "channel_source_fname" : "channel_source.csv", 
                    "print_dinf_drainage_area_raster" : "true",
                    "convert_csv_to_geojson" : "true"}

Create a driver object

In [ ]:
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "Diablo_COP30_UTM",
                                 write_prefix= "Diablo_COP30_UTM",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

Run *lsdtopotools*

In [ ]:
lsdtt_drive.run_lsdtt_command_line_tool()

## Look at the point data

In [ ]:
import pandas as pd
import geopandas as gpd
import cartopy as cp
import cartopy.crs as ccrs
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

Let's load the data using pandas. We then look to see what the column names are. 

In [ ]:
df = pd.read_csv("single_channel_nodes.csv")
list(df)

Oh, there are some stupid spaces in the column names. We can get rid of those with a `strip` command, like this:

In [ ]:
df = df.rename(columns=lambda x: x.strip())
list(df)

We can now convert into a geopandas dataframe

In [ ]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# We have to tell the geopandas data what geographic system we are in by using something called an EPSG code. 
# All major geographic projection and transformation system have this code. 
gdf.crs = "EPSG:4326" 

# The head command shows you what is in the file.
gdf.head()

## Making new data columns: slope and smoothed slope

Okay, we have flow distance and elevation in this file, but we also want to look at the slope of the channel. To get the slope, we need to calculate the change in elevation over the change in flow distance. The mathematical operation for this is called the gradient (or, if you want to use the notation of derivatives it is `dz/dx`).

The python package `numpy` has a built in function for calculating the gradient (`np.gradient`), which we use below to get the slope along the channel.

In [ ]:
z = gdf["elevation(m)"]
x = gdf["flow distance(m)"]
S = np.gradient(np.asarray(z),np.asarray(x))
gdf["slope"] = S
gdf.head()

Now lets plot the data

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]

# Now make channel profile plots
z = gdf["elevation(m)"]
x_locs = gdf["flow distance(m)"]
S = gdf.slope

# Create two subplots and unpack the output array immediately
plt.clf()
f, (ax1, ax2) = plt.subplots(2, 1)
ax1.scatter(x_locs, z,s = 0.2)
ax2.scatter(x_locs, S,s = 1)


ax1.set_xlabel("Distance from outlet ($m$)")
ax1.set_ylabel("elevation (m)")

ax2.set_xlabel("Distance from outlet ($m$)")
ax2.set_ylabel("Slope (m/m)")

plt.tight_layout()
plt.show()

This slope (bottom figure) is very noisy. One way to deal with this is to smooth the data. We can smooth the data by running a mobing window over it and doing some averaging inside the window. 

Python has lots of tools for this. In this case I use a `rolling` window and I have picked various settings. You don't need to worry about this too much, the only number that you might wanty to play with is the first number after `rolling` which is the number of datapoints in the window. The bigger this number, the more smoothed the data becomes. 

In [ ]:
gdf['slope_rolling'] = gdf.slope.rolling(40,win_type='hamming').mean()
gdf.head()

This plot will show the slope and the rolling slope, so you can see how the rolling window smooths the data. 

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

# Now make channel profile plots
# To get a single data column from a pandas dataframe (in this case called gdf_b2) you just put
# a full stop and then the name of the column
# If your column has spaces or funny characters in the name you need to use the square brackets like this:
# z = gdf_b2["elevation"]
# Which is an alternative way of isolating data
z = gdf["elevation(m)"]
x_locs = gdf["flow distance(m)"]
S = gdf.slope
SR = gdf.slope_rolling

# Create two subplots and unpack the output array immediately
plt.clf()
f, (ax1, ax2) = plt.subplots(2, 1)
ax1.scatter(x_locs, S,s = 1)
ax2.scatter(x_locs, SR,s = 1)


ax1.set_xlabel("Distance from outlet ($m$)")
ax1.set_ylabel("Slope (m/m)")

ax2.set_xlabel("Distance from outlet ($m$)")
ax2.set_ylabel("Rolling Slope (m/m)")

plt.tight_layout()

## Looking at the gradient and where the high gradient channels are along the channel profile

Okay, the rolling slope allows us to see some spikes in the gradient. Can we see this in the right places along the channel profile?

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]
# Now make channel profile plots
z = gdf["elevation(m)"]
x_locs = gdf["flow distance(m)"]
S = gdf.slope
SR = gdf.slope_rolling

# Create two subplots and unpack the output array immediately
plt.clf()
f, (ax1) = plt.subplots(1, 1)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

# Make the scatter plots
ax1.scatter(x_locs, z,s = 1, label='Longitudinal profile')
ax2.scatter(x_locs, SR,s = 1,c="r", label='Channel slope')

# Some code to make sure the legend renders on the same axis
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)


ax1.set_xlabel("Distance from outlet ($m$)")
ax1.set_ylabel("Elevation (m)")

ax2.set_xlabel("Distance from outlet ($m$)")
ax2.set_ylabel("Rolling Slope (m/m)")

plt.tight_layout()

## Saving the channel gradients to csv

I am afraid it is a little bit complicated to save the smoothed channel gradients to csv. 

Why? Becasue there are jumps in the flow distance at the tributary junctions. 

So to get the channel gradients we need to loop through each source key and get the gradients one by one. 

In [ ]:
# Now print to csv
gdf_b1.to_csv("diablo_channel_with_gradient.csv")